In [1]:
import os
import io
import re
import multiprocessing
import pickle

In [2]:
import neuralcoref
import spacy

print('spacy version:', spacy.__version__)
print('neuralcoref version:', neuralcoref.__version__)

spacy version: 2.1.3
neuralcoref version: 4.0.0


In [21]:
def write_data_to_disk(file, data):
    with open(file, 'wb') as fid:
        pickle.dump(data, fid)

# Reconhecimento de entidades

In [4]:
def read_file(path):
    try:
        with io.open(path, 'r', encoding='utf-8') as f:
            return f.read()
    except UnicodeDecodeError:
        try:
            with io.open(path, 'r', encoding='latin-1') as f:
                return f.read()
        except UnicodeDecodeError:
            return "Could not decode"

In [5]:
def extract_paragraphs(folder, file):
    path = os.path.join(folder, file) # name file

    if os.path.isfile(path):
        content = read_file(path)
        print(f"Content of book ... {file}")

        paragraphs = content.split('\n\n')

        return paragraphs
    return []

In [6]:
def new_paragraphs(paragraphs):
    paragraphs_new = []
    contents = []
    chapters = []
    old_idx = 0

    for paragraph in paragraphs:
        lines = paragraph.split('\n')

        if re.match('[IVXLCDM\d]+[\.]*', lines[0].upper().strip()):
            if not contents:
                contents = [" ".join(line.lower().split()) for line in lines]
                
                if not contents:
                    print("Not found content", id_file)
                    exit()
            continue

        if paragraph.lower().strip() in contents:
            if old_idx == 0:
                old_idx = len(paragraphs_new)
            else:
                current_idx = len(paragraphs_new)
                chapter = "\n\n".join(paragraphs_new[old_idx:current_idx])
                chapters.append(chapter)
                old_idx = current_idx
            continue

        if not re.match('.*[\w]+.*', paragraph):
            continue

        paragraphs_new.append(paragraph)

        if old_idx != 0:
            chapter = "\n\n".join(paragraphs_new[old_idx:])
            chapters.append(chapter)
        
    return paragraphs_new

In [7]:
def process_paragraph(paragraph, nlp):
    paragraph = paragraph.replace('\n', ' ')
    paragraph = " ".join(paragraph.split())
    doc = nlp(paragraph)
    doc_coref = doc._.coref_resolved
    doc_coref = nlp(doc_coref)

    entities = [ent.text for ent in doc_coref.ents if ent.label_ == "PERSON"]

    return entities

def extract_entities_from_paragraphs(paragraphs_to_process, nlp):
    """
    
    return:
    
    data: 
        (num paragraph, len_words, list_entities)
        
        [(5, 79, ['Kidd’s', 'Whoso']), ...]
    """
    
    data = []
    geral_entities = set()

    with multiprocessing.Pool() as pool:
        results = pool.starmap(process_paragraph, [(p, nlp) for p in paragraphs_to_process])

    for i, entities in enumerate(results):
        if entities:
            len_words = len(paragraphs_to_process[i].split())
            geral_entities.update(entities) # adiciona entities únicas
            data.append((i, len_words, entities))
            # print(i, len_words, entities)

    return data, list(geral_entities)

In [8]:
def all_entities(folder, file):     
    paragraphs = extract_paragraphs(folder, file)
    paragraphs_new = new_paragraphs(paragraphs)
    
    print('Número de parágrafos:', len(paragraphs))
    print('Número de parágrafos com personagens: ', len(paragraphs_new))     
    
    nlp = spacy.load("en_core_web_sm")
    neuralcoref.add_to_pipe(nlp)
    
    data, geral_entities = extract_entities_from_paragraphs(paragraphs_new, nlp)
    
    return data, geral_entities

In [18]:
def pk_data_entities(folder, df):
    
    files_ordered = sorted(os.listdir(folder), key=lambda name:int(name.split('_')[0]))
    
    for i in range(len(files_ordered)):
        file = files_ordered[i]
        print('\n')
        
        data, geral_entities = all_entities(folder, file)
        
        index = df["Index"][i]
        
        write_data_to_disk('dados/data_' + folder + '_' + str(index)+ '.pk', data)
        write_data_to_disk('dados/entities_' + folder + '_' + str(index)+ '.pk', geral_entities)

In [19]:
dfs = pd.read_csv('catalog_v0.xlsx - success.csv')

pk_data_entities(folder="success", df=dfs)



Content of book ... 1_BesideTheBonnieBrierBush_IanMaclaren_cleared.txt
Número de parágrafos: 1358
Número de parágrafos com personagens:  1158


Content of book ... 2_Trilby_GeorgeDuMaurier_cleared.txt
Número de parágrafos: 2407
Número de parágrafos com personagens:  1987


Content of book ... 3_TheAdventuresOfCaptainHorn_FrankRichardStockton_cleared.txt
Número de parágrafos: 1945
Número de parágrafos com personagens:  1608


Content of book ... 4_TheManxman_HallCaine_cleared.txt
Número de parágrafos: 5701
Número de parágrafos com personagens:  5227


Content of book ... 5_ThePrincessAline_RichardHardingDavis_cleared.txt
Número de parágrafos: 431
Número de parágrafos com personagens:  357


Content of book ... 6_TheMaster_IsraelZangwill_cleared.txt
Número de parágrafos: 3962
Número de parágrafos com personagens:  3511


Content of book ... 7_ThePrisonerOfZenda_AnthonyHope_cleared.txt
Número de parágrafos: 1762
Número de parágrafos com personagens:  1538


Content of book ... 8_Degener



Content of book ... 60_ThePrincessPasses_CNWilliamson_cleared.txt
Número de parágrafos: 2064
Número de parágrafos com personagens:  1688


Content of book ... 61_TheFightingChance_RobertWChambers_cleared.txt
Número de parágrafos: 4113
Número de parágrafos com personagens:  3865


Content of book ... 62_TheHouseOfAThousandCandles_MeredithNicholson_cleared.txt
Número de parágrafos: 2334
Número de parágrafos com personagens:  1983


Content of book ... 63_TheJungle_UptonSinclair_cleared.txt
Número de parágrafos: 1525
Número de parágrafos com personagens:  1289


Content of book ... 64_TheAwakeningOfHelenaRichie_MargaretDeland_cleared.txt
Número de parágrafos: 2595
Número de parágrafos com personagens:  2278


Content of book ... 65_TheSpoilers_RexBeach_cleared.txt
Número de parágrafos: 1997
Número de parágrafos com personagens:  1827


Content of book ... 66_TheBrassBowl_LouisJosephVance_cleared.txt
Número de parágrafos: 1937
Número de parágrafos com personagens:  1721


Content of book

In [20]:
dfo = pd.read_csv('catalog_v0.xlsx - other.csv')

pk_data_entities(folder="other", df=dfo)



Content of book ... 1_InTheLandOfCaveAndCliffDwellers_FrederickSchwatka_cleared.txt
Número de parágrafos: 301
Número de parágrafos com personagens:  191


Content of book ... 2_JudeTheObscure_ThomasHardy_cleared.txt
Número de parágrafos: 3689
Número de parágrafos com personagens:  3375


Content of book ... 3_TheGoldenAge_KennethGrahame_cleared.txt
Número de parágrafos: 522
Número de parágrafos com personagens:  425


Content of book ... 4_TheLostStradivarius_JohnMeadeFalkner_cleared.txt
Número de parágrafos: 361
Número de parágrafos com personagens:  208


Content of book ... 5_TheBritishBarbarians_GrantAllen_cleared.txt
Número de parágrafos: 412
Número de parágrafos com personagens:  361


Content of book ... 6_TheSorrowsOfSatan_MarieCorelli_cleared.txt
Número de parágrafos: 2412
Número de parágrafos com personagens:  1990


Content of book ... 7_RoseOfDutchersCoolly_HamlinGarland_cleared.txt
Número de parágrafos: 2561
Número de parágrafos com personagens:  2263


Content of book .



Content of book ... 60_LoveAmongChickens_PGWodehouse_cleared.txt
Número de parágrafos: 1674
Número de parágrafos com personagens:  1391


Content of book ... 61_TheFortunesOfPhilippa_AngelaBrazil_cleared.txt
Número de parágrafos: 685
Número de parágrafos com personagens:  547


Content of book ... 62_Pip_IanHay_cleared.txt
Número de parágrafos: 2199
Número de parágrafos com personagens:  1978


Content of book ... 63_TheMysteryOfTheYellowRoom_GastonLeroux_cleared.txt
Número de parágrafos: 1755
Número de parágrafos com personagens:  1530


Content of book ... 64_TheHillOfDreams_ArthurMachen_cleared.txt
Número de parágrafos: 457
Número de parágrafos com personagens:  370


Content of book ... 65_FatherAndSon_EdmundGosse_cleared.txt
Número de parágrafos: 532
Número de parágrafos com personagens:  330


Content of book ... 66_Jimbo_AlgernonBlackwood_cleared.txt
Número de parágrafos: 1074
Número de parágrafos com personagens:  950


Content of book ... 67_ThePromiseOfAmericanLife_HerbertD